# Import software libraries

In [1]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [2]:
# Connect to SQLite database.
conn=sqlite3.connect('data/prod_sample.db')
print("Database is connected : ",conn)

Database is connected :  <sqlite3.Connection object at 0x7fb2939c7030>


In [3]:
# List all the tables in the database.
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print('Tables in the database:')
for table in tables:
    print(table[0])

Tables in the database:
stock_description
online_retail_history


# Read data from the `online_retail_history` table

In [4]:
# Write the query to be executed that selects everything from the online_retail_history table.
query='SELECT * FROM online_retail_history'
# Use the read_sql function in pandas to read a query into a DataFrame.
online_retail_history=pd.read_sql(query,conn)
# Preview the first five rows of the data.
print(online_retail_history.head(5))

  Invoice StockCode  Quantity          InvoiceDate  Price CustomerID  \
0  536365    85123A         6  2010-12-01 08:26:00   2.55      u1785   
1  536367     84879        32  2010-12-01 08:34:00   1.69     u13047   
2  536373    85123A         6  2010-12-01 09:02:00   2.55      u1785   
3  536375    85123A         6  2010-12-01 09:32:00   2.55      u1785   
4  536378     20725        10  2010-12-01 09:37:00   1.65     u14688   

          Country  TotalAmount  
0  United Kingdom        15.30  
1  United Kingdom        54.08  
2  United Kingdom        15.30  
3  United Kingdom        15.30  
4  United Kingdom        16.50  


In [6]:
# Get the shape of the data.
print('Shape is ',online_retail_history.shape)

Shape is  (15321, 8)


# Read data from the `stock_description` table

In [5]:
# Write the query to be executed that selects everything from the online_retail_history table.
query='SELECT * FROM stock_description'

# Use the read_sql function in pandas to read a query into a DataFrame.
stock_description=pd.read_sql(query,conn)

# Preview the first five rows of the data.
print(stock_description.head(5))


  StockCode                  Description
0     10002  INFLATABLE POLITICAL GLOBE 
1     10080     GROOVY CACTUS INFLATABLE
2     10120                 DOGGY RUBBER
3    10123C        HEARTS WRAPPING TAPE 
4    10124A  SPOTS ON RED BOOKCOVER TAPE


In [7]:
# Get the shape of the data.
print(stock_description.shape)

(3952, 2)


# Aggregate the `online_retail_history` and `stock_description` datasets

In [8]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.
query = """
SELECT 
    online_retail_history.StockCode, 
    stock_description.Description, 
    online_retail_history.Invoice, online_retail_history.InvoiceDate 
FROM 
    online_retail_history
JOIN 
    stock_description 
ON 
    online_retail_history.StockCode = stock_description.StockCode
"""

# Use the read_sql function in pandas to read a query into a DataFrame.
aggregate=pd.read_sql(query,conn)

# Preview the first five rows of the data.
print(aggregate.head(5))


  StockCode                         Description Invoice          InvoiceDate
0    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536365  2010-12-01 08:26:00
1     84879       ASSORTED COLOUR BIRD ORNAMENT  536367  2010-12-01 08:34:00
2    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536373  2010-12-01 09:02:00
3    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536375  2010-12-01 09:32:00
4     20725             LUNCH BAG RED RETROSPOT  536378  2010-12-01 09:37:00


In [42]:
# Get the shape of the data.
print(aggregate.shape)

(17032, 9)

# Identify and fix corrupt or unusable data

In [11]:
# Check the value counts of the "Description" field.

print(aggregate['Description'].value_counts())

CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
REGENCY CAKESTAND 3 TIER              1711
?                                     1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64


In [12]:
# Remove rows where "Description" is just a question mark (?).
aggregate_cleaned=aggregate[aggregate.Description!='?']

# Preview the first five rows of the data.
print(aggregate_cleaned.head(5))


  StockCode                         Description Invoice          InvoiceDate
0    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536365  2010-12-01 08:26:00
1     84879       ASSORTED COLOUR BIRD ORNAMENT  536367  2010-12-01 08:34:00
2    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536373  2010-12-01 09:02:00
3    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536375  2010-12-01 09:32:00
4     20725             LUNCH BAG RED RETROSPOT  536378  2010-12-01 09:37:00


# Identify and remove duplicates

In [17]:
# Identify all duplicated data.
duplicated_data = aggregate.duplicated()

In [18]:
# Print the duplicated data.
print(aggregate[duplicated_data])


      StockCode                      Description Invoice          InvoiceDate
34        21212  PACK OF 72 RETROSPOT CAKE CASES  536464  2010-12-01 12:23:00
35        22197                   POPCORN HOLDER  536464  2010-12-01 12:23:00
110      85099B          JUMBO BAG RED RETROSPOT  536643  2010-12-02 11:57:00
149       84879    ASSORTED COLOUR BIRD ORNAMENT  536796  2010-12-02 15:46:00
178       20727          LUNCH BAG  BLACK SKULL.  536863  2010-12-03 11:19:00
...         ...                              ...     ...                  ...
16948     22423                                ?  581256  2011-12-08 11:21:00
16949     22423         REGENCY CAKESTAND 3 TIER  581256  2011-12-08 11:21:00
16972     20727          LUNCH BAG  BLACK SKULL.  581412  2011-12-08 14:38:00
16995     22423                                ?  581449  2011-12-08 17:37:00
16996     22423         REGENCY CAKESTAND 3 TIER  581449  2011-12-08 17:37:00

[374 rows x 4 columns]


In [20]:
# Remove the duplicated data.
cleaned_data=aggregate.drop_duplicates()

# Preview the first five rows of the data.
print(cleaned_data.head(5))


  StockCode                         Description Invoice          InvoiceDate
0    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536365  2010-12-01 08:26:00
1     84879       ASSORTED COLOUR BIRD ORNAMENT  536367  2010-12-01 08:34:00
2    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536373  2010-12-01 09:02:00
3    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536375  2010-12-01 09:32:00
4     20725             LUNCH BAG RED RETROSPOT  536378  2010-12-01 09:37:00


# Correct date formats

In [21]:
# Get the data types for every column in the DataFrame.
print(cleaned_data.dtypes)


StockCode      object
Description    object
Invoice        object
InvoiceDate    object
dtype: object


In [26]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.
cleaned_data['InvoiceDate']= pd.to_datetime(cleaned_data["InvoiceDate"], format='%Y-%m-%d')
print(cleaned_data)


      StockCode                         Description Invoice InvoiceDate
0        85123A  CREAM HANGING HEART T-LIGHT HOLDER  536365  2010-12-01
1         84879       ASSORTED COLOUR BIRD ORNAMENT  536367  2010-12-01
2        85123A  CREAM HANGING HEART T-LIGHT HOLDER  536373  2010-12-01
3        85123A  CREAM HANGING HEART T-LIGHT HOLDER  536375  2010-12-01
4         20725             LUNCH BAG RED RETROSPOT  536378  2010-12-01
...         ...                                 ...     ...         ...
17027     20727             LUNCH BAG  BLACK SKULL.  581538  2011-12-09
17028     20725             LUNCH BAG RED RETROSPOT  581538  2011-12-09
17029     22197                      POPCORN HOLDER  581579  2011-12-09
17030     20725             LUNCH BAG RED RETROSPOT  581583  2011-12-09
17031     84879       ASSORTED COLOUR BIRD ORNAMENT  581585  2011-12-09

[16658 rows x 4 columns]


In [27]:
# Get the data types for every column in the converted DataFrame.
print(cleaned_data.dtypes)


StockCode              object
Description            object
Invoice                object
InvoiceDate    datetime64[ns]
dtype: object


# Examine the table before finishing

In [28]:
# Preview the first five rows of the data.
print(cleaned_data.head(5))

  StockCode                         Description Invoice InvoiceDate
0    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536365  2010-12-01
1     84879       ASSORTED COLOUR BIRD ORNAMENT  536367  2010-12-01
2    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536373  2010-12-01
3    85123A  CREAM HANGING HEART T-LIGHT HOLDER  536375  2010-12-01
4     20725             LUNCH BAG RED RETROSPOT  536378  2010-12-01


# Load the dataset into a pickle file

In [29]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
cleaned_data.to_pickle('online_history_cleaned.pickle')
print("The dataset is saved as pickle")

The dataset is saved as pickle


In [30]:
# Close any connections to the database.

conn.close()
print("conncetions is closed")

conncetions is closed
